In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
print('Importing Librairies...')
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
%matplotlib inline
import time
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
print('Librairies imported')

___Author___='LumberJack Jyss'
print(" Fin'Lab Depedencies",Fore.BLUE,
      '\n Global Optimized LumberJack Environment Motor',Style.RESET_ALL,'\n GOLEM 55 \n LumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°',Style.RESET_ALL)
print(" INVESTING.COM'S COMMUNITY SENTIMENTS v1.10")
print('')

Importing Librairies...
Librairies imported
 Fin'Lab Depedencies  
 Global Optimized LumberJack Environment Motor  
 GOLEM 55 
 LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0° 
 INVESTING.COM'S COMMUNITY SENTIMENTS v1.10



In [3]:
try :
    sentiments_2018 = pd.read_csv('sentiments_2018.csv')
    compteur = pd.read_csv('compteur.csv')
    compteur = compteur.drop('Unnamed: 0',axis=1)
    sentiments_2018 = sentiments_2018.drop('Unnamed: 0',axis=1)

                        
except:

    sentiments = pd.read_csv('prepared_data.csv')
    sentiments = sentiments.drop('Unnamed: 0',axis=1)
    sentiments = sentiments.sort_values('Date')
    compteur = pd.read_csv('compteur.csv')
    compteur = compteur.drop('Unnamed: 0',axis=1)

    print(sentiments.iloc[18423,1][:4],sentiments.iloc[119263,1][:4])

    print(len(sentiments.Symbol.unique()))


    USER_ID = []
    DATE = []
    SYMBOL = []
    NAME = []
    SENTIMENT = []
    for i in range(18423,119263):
        print('\r Loop : ',i,'/119263',end='',flush=True)
        if sentiments.iloc[i,1][:4] == '2018':
            USER_ID.append(sentiments.iloc[i,0])
            DATE.append(sentiments.iloc[i,1])
            SYMBOL.append(sentiments.iloc[i,2])
            NAME.append(sentiments.iloc[i,3])
            SENTIMENT.append(sentiments.iloc[i,4])

    sentiments_2018 = pd.DataFrame()
    sentiments_2018['User ID'] = USER_ID
    sentiments_2018['Date'] = DATE
    sentiments_2018['Symbol'] = SYMBOL
    sentiments_2018['Name'] = NAME
    sentiments_2018['Sentiments'] = SENTIMENT

    sentiments_2018 = sentiments_2018.sort_values('User ID')

    print(len(sentiments_2018.Symbol.unique()))

    sentiments_2018.to_csv('sentiments_2018.csv')

In [4]:
try:
    megabase = pd.read_csv('megabase.csv')
except:
    megabase = pd.DataFrame()
    infrabase = pd.DataFrame()
    for boo in range(0,compteur.shape[0]):
        stock = compteur.iloc[boo,0]
        print('\r Scraping ',stock,' loop ',boo,'/',compteur.shape[0],end='',flush=True)
        try:
            infrabase = web.DataReader(stock, 'yahoo', '2018-01-01', '2019-02-28')
            infrabase['Stock'] = stock
            megabase = pd.concat((megabase,infrabase))
        except:
            print(Fore.RED,'Problème avec le ticker ',stock,Style.RESET_ALL)
            continue
    megabase.to_csv('megabase.csv')

In [5]:
megabase.head()

Date        High         Low        Open       Close     Volume  \
0  2018-01-02  237.070007  232.809998  235.779999  235.639999  2931000.0   
1  2018-01-03  235.729996  233.289993  235.070007  235.630005  2193700.0   
2  2018-01-04  239.440002  236.470001  237.000000  238.710007  2243100.0   
3  2018-01-05  240.899994  237.740005  238.649994  240.570007  1835900.0   
4  2018-01-08  240.940002  239.179993  239.380005  239.789993  1869000.0   

    Adj Close Stock  
0  223.990814   MMM  
1  223.981308   MMM  
2  226.909073   MMM  
3  228.677124   MMM  
4  227.935654   MMM

In [6]:
sentiments_2018.head()

User ID        Date Symbol            Name  Sentiments
0  43939137285  2018-03-22     FB    Facebook Inc           0
1  43939137285  2018-03-21     FB    Facebook Inc           1
2  43939137285  2018-03-21     FB    Facebook Inc           0
3  43939137285  2018-03-15   AMZN  Amazon Com Inc           0
4  43939137285  2018-01-10   ANSS           ANSYS           1

In [7]:
len(sentiments_2018['User ID'].unique())
    

33785

In [8]:
sentiments_USER = pd.DataFrame()
USER = []
SYMBOL = []
EVO = []
DATE = []
RANK = []

for coo in range(1,sentiments_2018.shape[0]):
    try:
        
        user = sentiments_2018.iloc[coo,0]

        a = 0
        if user == sentiments_2018.iloc[coo-1,0]:
            a+=1
            stock = sentiments_2018.iloc[coo,2]
            date = sentiments_2018.iloc[coo,1]
            date_22 = sentiments_2018.iloc[coo+30,1]
            if date_22 == '':
                date_22 = sentiments_2018.iloc[coo+31,1]
            if date_22 == '':
                date_22 = sentiments_2018.iloc[coo+32,1]
                
            perf = sentiments_2018.iloc[coo,-1]
            minibase = megabase.loc[megabase['Stock'] == stock]
            midibase = minibase.loc[minibase.Date.isin([date])]
            milibase = minibase.loc[minibase.Date.isin([date_22])]
            close = midibase.iloc[-1,6]
            close_22 = milibase.iloc[-1,6]
            evo = ((close_22 - close)/close)*100
            if (evo > 55 and perf == 1) or (evo < -55 and perf == 0):
                rank = 1
            else :
                rank = 0


            USER.append(user)
            DATE.append(date)
            SYMBOL.append(stock)
            EVO.append(evo)
            RANK.append(rank)

            #print('\r Loop : ',coo,' User : ',user,' Date : ',date,' Stock : ',stock,' Evolution : ',evo,' Rank : ',rank,end='',flush=True)
    except:
        #print(Fore.YELLOW,'\n Bon, bin, on va pas se mentir, ça foire au niveau du loop ',coo,' au niveau du ',stock,Style.RESET_ALL)
        #print(Fore.RED,'Le close_22 est à la date :',date_22,'\n',Style.RESET_ALL)
        continue

sentiments_USER['User'] = USER
sentiments_USER['Date'] = DATE
sentiments_USER['Symbol'] = SYMBOL
sentiments_USER['Evolution'] = EVO
sentiments_USER['Rank'] = RANK
print('Terminé!!!!!!!')
    

Terminé!!!!!!!


In [ ]:
megabase

In [9]:
2+2

4

In [12]:
sentiments_USER.sort_values('Rank',ascending=False)

User        Date Symbol   Evolution  Rank
24339  87837874815  2018-10-26     GE   59.683708     1
24202  87832443405  2018-01-19    AMD  155.837956     1
20897  87714075990  2018-02-09    AMD   97.082229     1
13419  87463149720  2018-11-30   NVDA   56.980256     1
32006  88064163990  2018-01-18    AMD   60.545309     1
46662  88518932565  2018-12-31   NVDA   99.740209     1
43190  88417269585  2018-11-19   NVDA   98.154794     1
43210  88417414005  2018-11-09     GE   61.514212     1
20947  87716266215  2018-02-22    AMD  171.874985     1
36145  88168825860  2018-09-28    AMD  -66.461639     1
31928  88062409200  2018-01-18     GE  -57.073907     1
1792   47862337470  2018-11-02     GE   77.042507     1
36161  88168902855  2018-07-30    AMD   67.713695     1
6164   87191558775  2018-12-26     GE   63.572319     1
36172  88169192565  2018-11-16     GE   75.189520     1
31907  88062092520  2018-10-24   MCHP   56.714296     1
16888  87593207760  2018-11-19     GE   60.081411     1
17001  87595849080  2018-06-08    AMD   83.999996     1
1893   47862337470  2018-10-19   XRAY   59.309434     1
44370  88457681520  2018-08-06    AMD   67.833248     1
24219  87832541280  2018-12-27   NVDA  108.627098     1
32108  88066869690  2018-11-26    AMD   61.503986     1
28225  87935374410  2018-12-20    APA   66.234619     1
43082  88414653060  2018-12-04   NVDA   57.308508     1
43084  88414653060  2018-11-29   NVDA   57.255270     1
32099  88066869690  2018-11-27     GE   84.132120     1
17073  87596721255  2018-12-19     GE   60.131216     1
5999   87183723120  2018-03-28    AMD   75.331295     1
9488   87300337050  2018-12-04   NKTR  121.052632     1
20845  87713575740  2018-10-09   AMAT   63.236896     1
...            ...         ...    ...         ...   ...
20129  87692096745  2018-05-03    MSI   15.948002     0
20127  87692096745  2018-06-05    AAL   -2.670878     0
20100  87692096745  2018-11-01    AMD  -18.397623     0
20126  87692096745  2018-05-03     HD    1.009883     0
20101  87692096745  2018-10-11     FB   25.816756     0
20102  87692096745  2018-02-27   MDLZ   -3.594274     0
20103  87692096745  2018-05-03      K    0.000000     0
20104  87692096745  2018-01-29     BA   -1.390515     0
20105  87692096745  2018-01-30    MCD   -3.731528     0
20106  87692096745  2018-05-25   AAPL    0.906773     0
20107  87692096745  2018-12-17     PG  -10.877184     0
20108  87692096745  2018-05-02    AAL   29.462203     0
20109  87692096745  2018-07-17   NFLX  -27.445457     0
20110  87692096745  2018-08-22     FB  -28.553330     0
20111  87692096745  2018-10-15    SYF   31.671192     0
20112  87692096745  2018-04-10    DAL   -0.038201     0
20113  87692096745  2018-11-01   AMZN   -5.055452     0
20114  87692096745  2018-10-15      T    3.284244     0
20115  87692096745  2018-10-18    DAL    3.976535     0
20116  87692096745  2018-02-06   ABBV   -9.721323     0
20117  87692096745  2018-02-27     PG   -8.522855     0
20118  87692096745  2018-03-05    XOM   13.241857     0
20119  87692096745  2018-06-08    SYF  -15.168666     0
20120  87692096745  2018-03-16   AAPL    7.958784     0
20121  87692096745  2018-05-09   MNST   18.652005     0
20122  87692096745  2018-04-19   AAPL    2.066511     0
20123  87692096745  2018-05-03    SYF  -19.688646     0
20124  87692096745  2018-10-05     PG  -10.514131     0
20125  87692096745  2018-07-20   ABBV    1.217521     0
59933  89128304055  2018-12-31   AMZN    0.000000     0

[59934 rows x 5 columns]

In [16]:
round((sentiments_USER['Rank'].sum()/sentiments_USER.shape[0])*100,2)

1.17

In [ ]:
sentiments.iloc[18423,1][:4],sentiments.iloc[119263,1][:4]

In [ ]:
sentiments_2018.head()

In [ ]:
TOTAL_STOCK = []
TOTAL_PERF = []
USERS = []
df_perf = pd.DataFrame()
for id_user in range(1,sentiments_2018.shape[0]):
    user = sentiments_2018.iloc[id_user,0]
    if user == sentiments_2018.iloc[id_user-1,0]:
        USERS.append(user)
        for nananere in range(0,compteur.shape[0]):
            #try:

            sentiments_STOCK_2018 = pd.DataFrame()
            DATE = []
            SYMBOL = []
            NAME = []
            SENTIMENT = []
            for i in range(0,sentiments_2018.shape[0]):
                if sentiments_2018.iloc[i,2] == stock:
                    DATE.append(sentiments_2018.iloc[i,0])
                    SYMBOL.append(sentiments_2018.iloc[i,1])
                    NAME.append(sentiments_2018.iloc[i,2])
                    SENTIMENT.append(sentiments_2018.iloc[i,3])

            #sentiments_STOCK_2018['Date'] = DATE
            #sentiments_STOCK_2018['Symbol'] = SYMBOL
            #sentiments_STOCK_2018['Name'] = NAME
            #sentiments_STOCK_2018['Sentiments'] = SENTIMENT

            sum_sent = sentiments_STOCK_2018.iloc[0,-1]
            a = 0
            signals_STOCK_2018 = pd.DataFrame()
            DATE = []
            SYMBOL = []
            NAME = []
            BULLISH = []
            BEARISH = []
            NUMBER = []
            for i in range(1,sentiments_STOCK_2018.shape[0]):
                print('\r Loop ',i,sentiments_STOCK_2018.shape[0]-1,end='')#,flush=True)
                a += 1
                if sentiments_STOCK_2018.iloc[i,0] == sentiments_STOCK_2018.iloc[i-1,0]:
                    sum_sent = sum_sent + sentiments_STOCK_2018.iloc[i,-1]
                else:
                    DATE.append(sentiments_STOCK_2018.iloc[i-1,0])
                    SYMBOL.append(sentiments_STOCK_2018.iloc[i,1])
                    NAME.append(sentiments_STOCK_2018.iloc[i,2])
                    BULLISH.append((sum_sent/a)*100)
                    BEARISH.append(100-((sum_sent/a)*100))
                    NUMBER.append(a)
                    sum_sent = sentiments_STOCK_2018.iloc[i,3]
                    a = 0
                    continue
            signals_STOCK_2018['Date'] = DATE
            signals_STOCK_2018['Symbol'] = SYMBOL
            signals_STOCK_2018['Name'] = NAME
            signals_STOCK_2018['Bullish'] = BULLISH
            signals_STOCK_2018['Bearish'] = BEARISH
            signals_STOCK_2018['# of signals'] = NUMBER



            start = signals_STOCK_2018.iloc[0,0]
            end = signals_STOCK_2018.iloc[-1,0]
            df = web.DataReader(stock, 'yahoo', start, '2019-02-28')

            #plt.figure(figsize=(20,10),dpi=300)
            #plt.title("Sentiment of users for Boeing's stocks - Year 2018")
            #plt.plot(df['Adj Close'],label='Close de INTC')
            #for foo in range(0,signals_STOCK_2018.shape[0]):
            #    if signals_STOCK_2018.iloc[foo,3] > 50:
            #        plt.axvline(x=signals_STOCK_2018.iloc[foo,0],color='green',lw=0.3)
            #    elif signals_STOCK_2018.iloc[foo,3] < 50:
            #        plt.axvline(x=signals_STOCK_2018.iloc[foo,0],color='red',lw=0.3)
            #    else:
            #        plt.axvline(x=signals_STOCK_2018.iloc[foo,0],color='black',lw=0.3)
            #plt.axvline(x=signals_STOCK_2018.iloc[-4,0],color='red',lw=0.01,label='Bearish Signal')
            #plt.axvline(x=signals_STOCK_2018.iloc[-5,0],color='black',lw=0.01,label='Neutral Signal')
            #plt.axvline(x=signals_STOCK_2018.iloc[0,0],color='green',lw=0.01,label='Bullish Signal')
            #
            #plt.legend()

            DELTA = []
            CLOSE = []
            CLOSE_22 = []
            CONFIRM = []
            for foo in range(0,signals_STOCK_2018.shape[0]):
                start = signals_STOCK_2018.iloc[foo,0]
                for poo in range(0,df.shape[0]):
                    if str(df.index[poo])[0:10] == start:
                        end = df.index[poo+30]#+dt.timedelta(30)
                        close = df.iloc[poo,5]
                        CLOSE.append(close)
                        for noo in range(0,df.shape[0]):
                            if df.index[noo] == end:
                                close_22 = df.iloc[noo,5]
                        CLOSE_22.append(close_22)
                        delta = round(((close_22 - close)/close)*100,2)
                        DELTA.append(delta)
                if start not in df.index:
                    DELTA.append('na')
                    CLOSE.append('na')
                    CLOSE_22.append('na')

            signals_STOCK_2018['Close'] = CLOSE
            signals_STOCK_2018['Close + 1M'] = CLOSE_22
            signals_STOCK_2018['% Variation'] = DELTA

            for goo in range(0,signals_STOCK_2018.shape[0]):

                if signals_STOCK_2018.iloc[goo,8] == 'na':
                    CONFIRM.append(0) 
                elif signals_STOCK_2018.iloc[goo,3] > 55 and signals_STOCK_2018.iloc[goo,8] > 10:
                    CONFIRM.append(1)
                elif signals_STOCK_2018.iloc[goo,4] > 55 and signals_STOCK_2018.iloc[goo,8] < 10:
                    CONFIRM.append(1)
                else:
                    CONFIRM.append(0)
            signals_STOCK_2018['Pertinence'] = CONFIRM
            perf = (sum(CONFIRM)/len(CONFIRM))*100
            print('\n La pertinence des signaux de ',Fore.YELLOW,stock,Style.RESET_ALL,' est de ',Fore.RED,round(perf,2),'%',Style.RESET_ALL)
            TOTAL_STOCK.append(stock)
            TOTAL_PERF.append(perf)
            #except :
             #   print('Problème dans la loop ',nananere)
              #  continue
        df_perf['Stock'] = TOTAL_STOCK
        df_perf['Perf'] = TOTAL_PERF

In [ ]:
df_perf = df_perf.sort_values('Perf',ascending=False)
df_perf.head()

In [ ]:
df_perf.to_csv('LesPerf.csv')